In [ ]:
from owlready2 import *
import pandas as pd

In [2]:
onto = get_ontology("/Users/sarahreb/Desktop/local/protege_files/cds-ontology.owl").load()

gndo = Namespace(
    world_or_ontology=onto,
    base_iri="https://d-nb.info/gnd/",
    name="gndo"
)

cds_docs = Namespace(
    world_or_ontology=onto,
    base_iri="https://constance-de-salm.de/archiv/#/document/",
    name="cds_docs"
)

In [ ]:
"""Individuals are instances in ontologies. They are created as any other Python instances. The first parameter is the name (or identifier) of the Individual; it corresponds to the .name attribute in Owlready2. If not given, the name if automatically generated from the Class name and a number.
"""

In [ ]:
# Parsing CSV file as DataFrame in order to create individuals.

df = pd.read_csv('../data/retrieved/filtered_cds_data.csv')
df = df.fillna(0)
df

In [6]:
# Adding instances of the CdS letters with senders and addressees.
index = 0
for url in df['URL']:
    new_letter = onto.Letter(re.sub('https://constance-de-salm.de/archiv/#/document/', '', url), namespace=cds_docs)
    new_letter.fud_key = df['FuD-Key'][index]
    new_letter.has_year.append(int(df['year'][index]))
    new_letter.has_decade.append(int(df['decade'][index]))
    new_letter.has_date.append(df['Datierung (JJJJ-MM-TT)'][index])
    new_sender = onto.Sender(re.sub('http://d-nb.info/gnd/', '', df['GND (Verfasser)'][index]), namespace=gndo)
    new_sender.label = df['Verfasser'][index]
    new_sender.has_viaf = df['VIAF (Verfasser)'][index]
    new_letter.has_sender.append(new_sender)
    if df['GND (Empfänger)'][index] == 0:
        new_addressee = onto.Sender(str(index), namespace=onto)
    else:
        new_addressee = onto.Sender(re.sub('http://d-nb.info/gnd/', '', df['GND (Empfänger)'][index]), namespace=gndo)
    new_addressee.label = df['Empfänger'][index]
    new_addressee.has_viaf = df['VIAF (Empfänger)'][index]
    new_letter.has_addressee.append(new_addressee)
    index += 1

In [ ]:
for i in onto.Letter.instances():
    #destroy_entity(i)
    print(i, i.has_date, i.fud_key, i.has_decade, i.has_year, i.has_sender, i.has_addressee)

In [9]:
onto.save(file = "cds_onto_test.owl", format = "rdfxml")